# **This is a brief introduction to the basics of how the codes works. You don't need to know all this to use the notebooks, but it is useful if you want to add or adjust anything to the code.**


## Run the following cell to import all the libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
from analysis_stm import MotionAnalyzer,ExpMetaData, DiffusionPlotter
import numpy as np
from scipy.optimize import curve_fit
import trackpy as tp
from PIL import Image
import matplotlib.pyplot as plt
from sxmreader import SXMReader
import matplotlib as mpl
import scipy
from sxmreader import SXMReader
from PIL import Image
from matplotlib.ticker import MultipleLocator
from matplotlib.animation import FuncAnimation, PillowWriter
import ipywidgets as widgets 
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np
import frame_correct as fc
import pims
import os
import yaml
import matplotlib as mpl

## The following cell records the name of the folder with our SXM files and the files we want. 

In [ ]:
folder_name = './test_sets/15K_3'
fileranges=range(6,35)
SXM_PATH = [folder_name + "/Image_{0:03}.sxm".format(i) for i in fileranges] 
for i in SXM_PATH:
    print(i)

## We then use the function SXMReader to read the files from the path. This function takes in a list of paths and outputs a sequence of frames.

In [ ]:
frames=SXMReader(SXM_PATH)
plt.imshow(frames[0]) #plots the first frame
plt.show()

## SXMReader has multiple options to correct our image, but we mainly use 'lines' which subtracts linear fits from each row. 

In [ ]:
frames=SXMReader(SXM_PATH,correct='lines') #Uses lines correction
plt.imshow(frames[0])
plt.show()

## Sometimes our data has tears in it due to some issue during the image taking. We correct those by averaging the nearest lines.

In [ ]:
frames=SXMReader(SXM_PATH,correct='lines')
fig,ax=plt.subplots()
plt.title('Frame 2 Before Correction')
ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(5))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1)) 
ax.imshow(frames[2]) # frame with a tear 
correction=[2,107] #tear row
frame_new=np.array(frames[correction[0]])
frame_new[correction[1],:]=(frame_new[correction[1]+1,:]+frame_new[correction[1]-1,:])/2 #mean of adjacent frames
frame_new = (frame_new - frame_new.min()) / (frame_new.max() - frame_new.min())  # scale to [0, 1]
frame_new = frame_new * 2 - 1    
frame_new = pims.Frame(frame_new, frame_no=correction[0])
frames[correction[0]]=frame_new
fig,ax=plt.subplots()
plt.title('Frame 2 After Correction')

plt.imshow(frames[2])
plt.show()

## We then want to correct for any movements of the scan area. The first option requires a visible background like a moire (or potentially not using lines correction, but this is not well tested). 

The functions in frame_correct.py are for this purpose. Run the next two cells to see how this correction looks. It might take some time depending on your computer (it's not that well optimized). The second option is just subtracting ensemble particle drift. This works even when you are on monolayer with a flat background, but is not as accurate.

In [ ]:
import frame_correct as fc
from collections import namedtuple

#These are parameters for trackpy to do molecule search 
params = {
    'molecule_size': 7,
    'min_mass': 2.1,
    'max_mass': 100,
    'min_size': 1,
    'max_ecc': 1,
    'separation': 1,
    'search_range': 30,
    'adaptive_stop': 1,
    'diffusion_time': 10,
    'threshold': 0.12,
}

# This stuff is just to get the formatting consistent with how the function uses it
Params = namedtuple(
                'Params', 
                ['molecule_size', 
                    'min_mass',
                    'max_mass',
                    'separation',
                    'min_size',
                    'max_ecc',
                    'adaptive_stop',
                    'search_range',
                    'threshold',
                    'diffusion_time',])
PARAMS = Params(**params)
from types import SimpleNamespace
par=SimpleNamespace(**params)
frame_drift_par={
                   'steps':2,
                   'adjust':[[],[]],
                   'output_crop':200,                      
}
frames_ = SXMReader(SXM_PATH, correct='lines')
frames_all=[]
frames=[]
for frame in frames_:
    frames.append(frame)
frames_=frames
correction=[2,107]
img=np.array(frames_[correction[0]])
img[correction[1],:]= (img[correction[1]+1,:]+img[correction[1]-1,:])/2
img = (img - img.min()) / (img.max() - img.min())  # scale to [0, 1]
img = img * 2 - 1    
img = pims.Frame(img, frame_no=correction[0])
frames_[correction[0]]=img
plt.show()
frames_corrected=fc.Frame_correct_loop(frames_[0:],PARAMS,**frame_drift_par)

In [ ]:

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()
fig=plt.figure(figsize=(8, 6), dpi=120)
ax1=plt.axes(xlim=(0, 256), ylim=(0, 256), frameon=False)
plt.axis('off')
ln, = ax1.plot([], [], lw=3)
#ax1.set_prop_cycle(color=['g'])
def animate(i):
    plt.cla()
    plt.title(i)
    ax1.set_prop_cycle(color=['g', 'r', 'c', 'm', 'y', 'k'])
    plt.imshow(frames_corrected[i], cmap='gray',zorder=0)

# Set up formatting for the movie files
#Writer = matplotlib.animation.writers['ffmpeg']
#writer = Writer(fps=5, metadata=dict(artist='Me'), bitrate=1800)
line_ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(frames_corrected))
#line_ani.save('18K_manual.mp4', writer=writer)
line_ani

You might notice some slight (or major) shifts that are not corrected. Ideally, changing the various parameters 
can help fix this. 

## We then use trackpy to extract particle data from these images.
We added rotation tracking as well, which is why we use a custom trackpy from my github rather than just the normal one. Below are the most useful trackpy functions

In [ ]:
f=tp.locate(frames_[0], #inputs an image array and search parameters, returns a a dataframe with position, size, angle of molecules in pixels
          diameter=params['molecule_size'],
          minmass= params['min_mass'],
          separation= params['separation'],
          threshold= params['threshold'])


In [ ]:
t=tp.batch(frames_, #same as trackpy.locate but for multiple frames
          diameter=params['molecule_size'],
          minmass= params['min_mass'],
          separation= params['separation'],
          threshold= params['threshold'])
tp.link(t, #tracks particles between multiple frames using data from tp.batch
        search_range=params['search_range'],
       adaptive_stop=params['adaptive_stop'])

In [ ]:
tp.annotate(f,frames_[0]) #Visualizes the paticles selected with tp.locate

## The code we use is stored in analysis_stm.py, which does the following:

1. uses sxmreader to convert sxm files to readable format
2. optionally uses frame_correct to stabilize image
3. uses trackpy to extract particle positions
4. performs calculations for properties such as diffusion constant